# Analyzing borrowers’ risk of defaulting


The goal of the project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a credit scoring of a potential customer. A credit scoring is used to evaluate the ability of a potential borrower to repay their loan.

Description of the data:
- children — the number of children in the family
- days_employed — how long the customer has been working
- dob_years — the customer’s age
- education — the customer’s education level
- education_id — identifier for the customer’s education
- family_status — the customer’s marital status
- family_status_id — identifier for the customer’s marital status
- gender — the customer’s gender
- income_type — the customer’s income type
- debt — whether the customer has ever defaulted on a loan
- total_income — monthly income
- purpose — reason for taking out a loan

## Step 1. General overview of data 

In [1]:
#Read initial file

import pandas as pd
data = pd.read_csv('/Users/irinaarmasheva/Desktop/ЯНДЕКС_ПРАКТИКУМ/Проекты/Предобработка_данных/data.csv')


In [2]:
#Check information about the dataset

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#Rename column 'dob_years' to 'full_years'

data = data.rename(columns={'dob_years': 'full_years'})

In [4]:
#Check first 5 rows of the dataset

display(data.head())

,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
#Check the main statistics with the describe() method

data.describe()

,children,days_employed,full_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
#Check total quantity of negative values in column days_employed

print(data[data['days_employed'] < 0]['income_type'].count())

15906


In [7]:
#Calculate min and avg for days_employed column depending on income type 

display(data.groupby('income_type').agg({'days_employed' : ['min', 'mean']}))


days_employed               
                           min           mean
income_type                                  
безработный      337524.466835  366413.652744
в декрете         -3296.759962   -3296.759962
госслужащий      -15193.032201   -3399.896902
компаньон        -17615.563266   -2111.524398
пенсионер        328728.720605  365003.491245
предприниматель    -520.848083    -520.848083
сотрудник        -18388.949901   -2326.499216
студент            -578.751554    -578.751554

**Conclusion**

There are missed values in columns days_employed and total_income that need to be filled.

It is decided to rename column dob_years into full_years, the rest of the columns are named correctly stylistically and by meaning.

Type of data in columns total_income and days_employed is float, it is preferred to change to int.

Column days_employed contains negative values that need to be replaced. Total quantity of rows with negative values is 
15906 and we cannot just delete it. The assumption is that days-employed is calculated as a difference between employment date and date of data extraction (it is necessary to discuss with developers). For categories retiree and unemployed the value in column days_employed is unrealistically big.  


## Step 2. Data preprocessing

### Processing of missed values

In [8]:
#Check first 10 rows

display(data[data['days_employed'].isnull()].head(10))


,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [9]:
#Replace negative values in column days_employed by positive values

data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'].abs()

In [10]:
#Check the result by looking at avg values by category

print(data.groupby('income_type')['days_employed'].mean())

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          3399.896902
компаньон            2111.524398
пенсионер          365003.491245
предприниматель       520.848083
сотрудник            2326.499216
студент               578.751554
Name: days_employed, dtype: float64


In [11]:
#Check how many values are missed in columns 'days_employed' и 'total_income'

print(len(data[data['days_employed'].isnull()]))
print(len(data[data['total_income'].isnull()]))

2174
2174


In [12]:
#Replace values in column days_employed for retiree and unemployed by avg values in this column:
days_employed_avg = data['days_employed'].mean()

data.loc[(data['income_type'] == 'пенсионер'), 'days_employed'] = days_employed_avg
data.loc[(data['income_type'] == 'безработный'), 'days_employed'] = days_employed_avg


In [13]:
#Check the result

print(data[data['income_type'] == 'пенсионер']['days_employed'].mean())
print(data[data['income_type'] == 'безработный']['days_employed'].mean())

66914.72890681757
66914.72890682195


In [14]:
#Check count of values by income_type

print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


In [15]:
#Unite small categoies into a new category ('other)')

data.loc[data['income_type'] == 'предприниматель', 'income_type'] = 'компаньон'
data.loc[data['income_type'] == 'безработный', 'income_type'] = 'другое'
data.loc[data['income_type'] == 'в декрете', 'income_type'] = 'другое'
data.loc[data['income_type'] == 'студент', 'income_type'] = 'другое'

In [16]:
#Check updated income_type categories

print(data['income_type'].value_counts())

сотрудник      11119
компаньон       5087
пенсионер       3856
госслужащий     1459
другое             4
Name: income_type, dtype: int64


In [17]:
#Calculate median and avg for column 'total_income' depending on category

print(data.groupby('income_type').agg({'total_income': ['mean','median']}))


              total_income               
                      mean         median
income_type                              
госслужащий  170898.309923  150447.935283
другое       103677.564849   79079.308649
компаньон    202482.281401  172396.000846
пенсионер    137127.465690  118514.486412
сотрудник    161380.260488  142594.396847


In [18]:
#Calculate median and avg for column  'days_employed' depending on category

print(data.groupby('income_type').agg({'days_employed': ['mean','median']}))

            days_employed              
                     mean        median
income_type                            
госслужащий   3399.896902   2689.368353
другое       34426.242332  35105.744434
компаньон     2111.176937   1546.333214
пенсионер    66914.728907  66914.728907
сотрудник     2326.499216   1574.202821


In [19]:
#Check which categories have missed values in column 'total_income'

print(data[data['total_income'].isnull()].groupby('income_type')['income_type'].count())

income_type
госслужащий     147
компаньон       509
пенсионер       413
сотрудник      1105
Name: income_type, dtype: int64


In [20]:
#Check which categories have missed values in column 'days_employed'

print(data[data['days_employed'].isnull()].groupby('income_type')['income_type'].count())

income_type
госслужащий     147
компаньон       509
сотрудник      1105
Name: income_type, dtype: int64


In [21]:
#Replace missed values in column 'total_income' by median value depending on category

med = data.groupby('income_type')['total_income'].transform('median')
data['total_income'] = data['total_income'].fillna(med)


In [22]:
#Check if there is missed values in column 'total_income'

data['total_income'].isna().sum()

0

In [23]:
#Replace missed values in column 'days_employed' by median value depending on category

med = data.groupby('income_type')['days_employed'].transform('median')
data['days_employed'] = data['days_employed'].fillna(med)


In [24]:
#Check if there are some remaining values in a dataset

data.isna().sum()

children            0
days_employed       0
full_years          0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [25]:
#Check unique values and ther quantity in column debt

data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [26]:
#Check unique values and ther quantity in column children

data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [27]:
# Replace value -1 by 1 and 20 by 2 assuming that it was a manual mistake

data['children'] = data['children'].replace(to_replace=[-1,20],value=[1,2])
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Conclusion**

Negative values in column days_employed are replaced by positive using abs (method).  
Missed values in columns 'days_employed' and 'total_income' were replaced by median values for each category.  
A new category was introduced instead of small categories (less than 3 records).  
Values -1 and 20 were replaced by 1 and 20 in column children.


### Change type of data

In [28]:
#Change data type in column 'total_income' to int

data['total_income'] = data['total_income'].astype('int')

In [29]:
#Change data type in column 'days_employed' to int

data['days_employed'] = data['days_employed'].astype('int')

In [30]:
#Check data info

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   full_years        21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Processing of duplicates

In [31]:
#Check total quantity of duplicates in a dataset

data.duplicated().sum()

54

In [32]:
#Check categories in column 'education'

data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [33]:
#Change register in column education to lower

data['education'] = data['education'].str.lower()


In [34]:
#Check duplicates in column education after register was changed 

data.duplicated().sum()

71

In [35]:
#Check categories in column 'family_status'

data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [36]:
#Check categories in column 'purpose'

data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [37]:
#Delete duplicates

data = data.drop_duplicates().reset_index(drop=True)

In [38]:
#Check the result

data.duplicated().sum()

0

**Conclusion**

Letter case was changed to lower for records in column 'education'. As the result 71 duplicates were found. These duplicates were deleted.  
Besides, in column 'purpose' there are records that are similar by meaning but input with different wording.

### Lemmatization

In [39]:
#Import library for lemmatization

from pymystem3 import Mystem
m = Mystem() 

In [40]:
#Lemmatize values in column purpose
text = data['purpose'].unique()
text = ' '.join(text)

lemmas = m.lemmatize(text)
print(lemmas) 

                

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

In [41]:
#Count frequency of each word in column purpose

from collections import Counter
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


**Conclusion**

Library pymystem3 was used for lemmatization.

As the result, the following categories were defined as loan purpose:
property, auto, education, wedding.


### Data categorization

In [42]:
# Create a formular to categorize loan purpose depending on a defining word in a cell. 

def purposes_dict (purpose):
    """
    Return category of a loan purpose depending on a defining word in an initial goal.
    If initial purpose contains 'автомобил' - category 'автомобиль' is assigned
    If initial purpose contains 'свадьб' - category 'свадьба' is assigned
    If initial purpose contains 'образован' - category 'образование' is assigned
    If initial purpose contains 'недвижимост' - category 'недвижимость' is assigned
    If initial purpose contains 'жиль' - category 'недвижимость' is assigned
    """
    
    if 'автомобил' in purpose:
        return 'автомобиль'
    if 'свадьб' in purpose:
        return 'свадьба'
    if 'образован' in purpose:
        return 'образование'
    if 'недвижимост' or 'жиль' in purpose:
        return 'недвижимость'
    
    
data['purpose_group'] = data['purpose'].apply(purposes_dict)
display(data.head(10))

,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,66914,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [43]:
#Check the result

print(data['purpose_group'].isna().sum())

0


In [44]:
#Сheck the spread of the income category

data_min = data['total_income'].min()
data_quantile_lower = data['total_income'].quantile(0.33)
data_median  = data['total_income'].median()
data_quantile_upper = data['total_income'].quantile(0.66)
data_max = data['total_income'].max()


print(data_min)
print(data_quantile_lower)
print(data_median)
print(data_quantile_upper)
print(data_max) 

20667
118514.0
142594.0
172396.0
2265604


In [45]:
#Create formula for income categorization

def income_dict (total_income):
    """"
    Return category depending on income
    If income is  < 1st quantile - return 'низкий'
    If income is >= 1st quantile and < 2nd quantile - return 'средний'
    If income is >= 2nd quantile - return 'высокий'
    """
    if total_income < data_quantile_lower:
        return 'низкий'
    if data_quantile_lower <= total_income < data_quantile_upper:
        return 'средний'
    if total_income >= data_quantile_upper:
        return 'высокий'
    
data['income_group'] = data['total_income'].apply(income_dict)
display(data.head(10))    

,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,66914,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,средний


In [46]:
#Check that for each row an income category is assigned

print(data['income_group'].isna().sum())

0


**Conclusion**

As the result of lemmatization, 4 main categories of loan purpose were detected: property, auto, ediucation, wedding.

Also, total income was categorized as low, medium, high.



## Step 3. Answer the question

- Is there a dependency between children and debt payment?

In [47]:
#Create a table with debt counted depending on quanity of children

data_pivot_children = data.pivot_table(index=['children'], columns = 'debt', values = 'income_type', aggfunc = 'count')

data_pivot_children['ratio'] = data_pivot_children[0] / (data_pivot_children[1] + data_pivot_children[0])
print(data_pivot_children.sort_values(by = 'ratio', ascending = False))

debt            0       1     ratio
children                           
0         13028.0  1063.0  0.924562
3           303.0    27.0  0.918182
1          4410.0   445.0  0.908342
2          1926.0   202.0  0.905075
4            37.0     4.0  0.902439
5             9.0     NaN       NaN


**Conclusion**

The table shows, that people without children borrow money more often and have the highest ration of loan return.  
People with 3 chidren follow them.  
People with 5 children are the smallest group of borrows (probably due to the fact that to have 5 children is rare in general) who always return their debt.  
People with 1, 2 and 4 children have approximately the same ration of debt return.  

- Is there a dependency between family status and debt payment

In [48]:
#Create a table with debt counted depending on family status

data_pivot_status = data.pivot_table(index=['family_status'], columns = 'debt', values = 'income_type', aggfunc = 'count')

data_pivot_status['ratio'] = data_pivot_status[0] / (data_pivot_status[0] + data_pivot_status[1])
print(data_pivot_status.sort_values(by = 'ratio', ascending = False))

debt                       0    1     ratio
family_status                              
вдовец / вдова           896   63  0.934307
в разводе               1110   85  0.928870
женат / замужем        11408  931  0.924548
гражданский брак        3763  388  0.906529
Не женат / не замужем   2536  274  0.902491


**Conclusion**

Data shows that debt return ratio is higher for widows and divorced people (assumption).  
The highest number of non-payer is among those who are in a common law marriage or have never been married.


- Is there a dependency between total income and debt payment?

In [49]:
#Create a table with debt counted depending on income group

data_pivot_income = data.pivot_table(index=['income_group'], columns = 'debt', values = 'income_type', aggfunc = 'count')
data_pivot_income['ratio'] = data_pivot_income[0] / (data_pivot_income[1] + data_pivot_income[0])
print(data_pivot_income.sort_values(by = 'ratio', ascending = False))

debt             0    1     ratio
income_group                     
высокий       6958  558  0.925758
низкий        6165  544  0.918915
средний       6590  639  0.911606


**Conclusion**

The highest ration of poeple who pay their debt on time is in a group with high income.  
Suprisingly, people low incme are on the second place in terms of loan payment. Probably, finacial discipline and planning help to manage their loan more efficiently.  
People are with medium income are on the third place.  


- Is there a dependency between loan purpose and debt payment?

In [50]:
#Create a table with debt counted depending on purpose group

data_pivot_purpose = data.pivot_table(index=['purpose_group'], columns = 'debt', values = 'income_type', aggfunc = 'count')
data_pivot_purpose['ratio'] = data_pivot_purpose[0] / (data_pivot_purpose[1] + data_pivot_purpose[0])
print(data_pivot_purpose.sort_values(by = 'ratio', ascending = False))

debt               0    1     ratio
purpose_group                      
недвижимость   10029  782  0.927666
свадьба         2138  186  0.919966
образование     3643  370  0.907800
автомобиль      3903  403  0.906410


**Conclusion**

People who take a loan for buying a property are the largest group and the most disciplined in terms of debt repayment.
People who take a loan for wedding organization are on the second place.
People who take a loan for eduction are on the third place. Most likely major part of the borrows in the group are young people who are at higher risk of not paying their debt.
People who take a loan for a car have the lowest ratio of debt return.


## Step 4. General conclusion

In the project the data was preprocessed: missing values were filled with mean and median values, duplicates were deleted and values in columns 'purpose' and 'income_type' were categorized for further analysis.

As the result of analysis it was revealed that an ideal borrower is a widow or divorced person without children with high income who buys a property.  
The most risky group is people that have never been married with 4 children, medium income, who buying a new car.

The results of the project can be used for building a scoring system of a potential customer.
